# NPS prediction

**Net promoter score (NPS)** is a single survey question asking, to rate the **likelihood** that they would recommend a company. In this project we are going to use time-series prediction in order to predict the **monthly** and **quarterly** NPS of Santalucia (Spanish Insurence company)
 

###  1.0 Importing Data From SantaLucia

In [ ]:
#import arima model
from statsmodels.tsa.arima_model import ARIMA
#import graph for missing values
import missingno as msno
from functools import reduce
# Import Principal Component Analysis from sklearn
from sklearn.decomposition import PCA
# Import train test split
from sklearn.model_selection import train_test_split
#Import preprocessing from sklearn
from sklearn import preprocessing 
#Impor graphing library seaborn
import seaborn as sns


In [ ]:
#import basic libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd

#import profiling library for profiling
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
import pandas_profiling as pp
from pandas_profiling import ProfileReport

#------------Not necessary---------------
from google.colab import drive
drive.mount('/content/drive')
#------------------------------------------


     - 25.9 MB 802 kB/s
     |████████████████████████████████| 10.1 MB 5.9 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
     |████████████████████████████████| 102 kB 7.6 MB/s 
     |████████████████████████████████| 675 kB 64.3 MB/s 
     |████████████████████████████████| 4.7 MB 56.3 MB/s 
     |████████████████████████████████| 62 kB 823 kB/s 
     |████████████████████████████████| 812 kB 66.0 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-3.1.1-py2.py3-none-any.whl size=261287 sha256=dee468e64ffa17487341446ff6266fa8f2da154de03f0cb808afa8f8741fcf62
  Stored in directory: /tmp/pip-ephem-wheel-cache-cd6epbu4/wheels/cc/d5/09/083fb07c9363a2f45854b0e3a7de7d7c560f07da74b9e9769d
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=71695387823185c5824537712bf4b4638c05d9a3eae53516f6edf20e9d367457
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d

Mounted at /content/drive


In [ ]:
#Path of the document 
ruta = "drive/MyDrive/Clases/Estudiantes/Noelia/Regresion_Lineal/BaseSTL-SAT-2016-2021_v4.xlsx"
d2 = pd.read_excel(ruta)

In [ ]:
df = d2

In [ ]:
#Separate into numerical variables
variables_numericas = ["Antiguedad","EDAD","N_SINIESTROS","NSIN_AFP","NSIN_DEC","NSIN_HOG","NSIN_SLD",
                       "NSIN_VIA","NSIN_VIR","NUM_POL_AFP","NUM_POL_DEC","NUM_POL_HOG","NUM_POL_LSLD","NUM_POL_TOTAL",
                       "NUM_POL_VIA","NUM_POL_VIR","PUNTUACION","AGENCIA","CUANTIA","MODALIDAD","RAMO","RAMO1","RECO",
                      "SINIESTROS","TRIM","FECHA_ENCUESTA"]

In [ ]:
#Separate into cualitative variables
variables_cualitativas = ["AREA","PROVINCIA",
                                      "VINCULACION","sexolis","DESC_MODALIDAD"]

In [ ]:
df_variables_numericas = df[variables_numericas]
df_variables_cualitativas = df[variables_cualitativas]

#### 1.1 Load NPS

In [ ]:
df_nps_stl = pd.read_excel(ruta, sheet_name="NPS_mes")

In [ ]:
df_nps_stl = df_nps_stl.rename(columns={"ANIO_MES": "FECHA_ENCUESTA", "%_NPS": "Y"})

In [ ]:
df_nps_stl.dtypes

Etiquetas de fila                object
FECHA_ENCUESTA                   object
Promedio de PROM (PONDERADO)    float64
Promedio de NEU (PONDERADO      float64
Promedio de DET (PONDERADO)     float64
NPS                             float64
dtype: object

In [ ]:
df_nps_stl["FECHA_ENCUESTA"] = pd.to_datetime(df_nps_stl["FECHA_ENCUESTA"], format="%Y-%m")

In [ ]:
df_nps_stl.dtypes

Etiquetas de fila                       object
FECHA_ENCUESTA                  datetime64[ns]
Promedio de PROM (PONDERADO)           float64
Promedio de NEU (PONDERADO             float64
Promedio de DET (PONDERADO)            float64
NPS                                    float64
dtype: object

In [ ]:
nps_df = df_nps_stl

In [ ]:
nps_df

,Etiquetas de fila,FECHA_ENCUESTA,Promedio de PROM (PONDERADO),Promedio de NEU (PONDERADO,Promedio de DET (PONDERADO),NPS
0,201601.Enero,2016-01-01,0.488499,0.320136,0.185113,0.303386
1,201602.Febrero,2016-02-01,0.490933,0.321561,0.181199,0.309734
2,201603.Marzo,2016-03-01,0.508864,0.269049,0.225050,0.283814
3,201604.Abril,2016-04-01,0.529178,0.319789,0.152801,0.376378
4,201605.Mayo,2016-05-01,0.438591,0.385417,0.168912,0.269680
...,...,...,...,...,...,...
66,202107.Julio,2021-07-01,0.492255,0.327343,0.173006,0.319249
67,202108.Agosto,2021-08-01,0.613841,0.174724,0.199610,0.414231
68,202109.Septiembre,2021-09-01,0.534032,0.210234,0.254238,0.279794
69,202110.Octubre,2021-10-01,0.548107,0.257555,0.193531,0.354576


### 2.0 Exploratory Data Analysis

In [ ]:
profile = ProfileReport(df_variables_numericas, title="Pandas Profiling Report", explorative=True)

In [ ]:
profile.to_widgets()

/usr/local/lib/python3.7/dist-packages/pandas_profiling/profile_report.py:406: UserWarning: Ipywidgets is not yet fully supported on Google Colab (https://github.com/googlecolab/colabtools/issues/60).As an alternative, you can use the HTML report. See the documentation for more information.
  "Ipywidgets is not yet fully supported on Google Colab (https://github.com/googlecolab/colabtools/issues/60)."
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

### 3.0 Data Processing



$$ NPS = promotores_p - detractores_p $$

The equation of the NPS is the number of promoters subtracted by the number of detractors

In [ ]:
#Selecting the years
df["Año"] = df["FECHA_ENCUESTA"].dt.year

In [ ]:
#Selecting the months
df["Mes"] = df["FECHA_ENCUESTA"].dt.month

In [ ]:
#Selecting the collection of months
df["Año-Mes"] = pd.to_datetime(df['FECHA_ENCUESTA']).dt.to_period('M')

In [ ]:
df

,FECHA_ENCUESTA,SOCIEDAD,AREA,AGENCIA,PROVINCIA,SAT1,REC,NPS,AGE01,AGE03,AGE04,AGE05,AGESG,Antiguedad,AUT,CAT01,CAT03,CAT04,CAT05,CATSG,CL01,CL02,CL02_cod,NOMBRE,Apellido1,Apellido2,CODSTIGA,COMP01,COMP021,COMP021SG,COMP021SGMOT,COMP021SGMOT_1,COMP022,COMP022SG,COMP022SGMOT,COMP022SGMOT_2,COMP023,COMP023SG,COMP023SGMOT,COMP023SGMOT_3,...,NSIN_SLD,NUM_POL_AFP,NUM_POL_DEC,NUM_POL_HOG,NUM_POL_VIA,NUM_POL_VIR,NUM_POL_LSLD,NUM_POL_TOTAL,OFE01,OFESG,PRO01,PRO02,PROSG,PUB01,PUBSG,PUNTUACION,RAMO,RAMO1,RECO,SAT2,sexolis,SIN01,SINIESTROS,SINSG,SUG,telefono1,telefono2,VINCULACION,WEB01,WEB02,WEB03,WEB04,WEBSG,MES,TRIM,ANO,PESO,Año,Mes,Año-Mes
0,2016-01-25 00:00:00,NaN,CTAS 4,2.0,Madrid,10.0,10.0,Promotor,2,NaN,NaN,NaN,NaN,1,1.0,2,NaN,NaN,NaN,NaN,2.0,26.0,1.0,Alina Maria,Brasoveanu,NaN,STL-SAT-0116-27609,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1,0,0,0,0,0,2,2.0,NaN,10.0,10.0,10.0,1,8.0,54.6,MULTI,NaN,10.0,10.0,M,NaN,NO,NaN,NaN,675170658,NaN,02.Baja,NaN,NaN,NaN,NaN,NaN,01.Enero,1T-2016,2016,1.667358,2016,1,2016-01
1,2016-01-21 00:00:00,NaN,AMSUR 1,536.0,Málaga,10.0,10.0,Promotor,2,NaN,NaN,NaN,NaN,1,1.0,2,NaN,NaN,NaN,NaN,2.0,27.0,1.0,Claudia Lucía,Espérguez,Miranda,STL-SAT-0116-27194,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,0,0,0,0,0,1,1,2.0,NaN,10.0,9.0,10.0,1,9.0,73.5,MONO,NaN,10.0,9.0,M,1.0,SI,10.0,NaN,649534235,NaN,03.Media,NaN,NaN,NaN,NaN,NaN,01.Enero,1T-2016,2016,0.419021,2016,1,2016-01
2,2016-01-20 00:00:00,NaN,CTAS 1,172.0,Madrid,10.0,10.0,Promotor,2,NaN,NaN,NaN,NaN,5,1.0,2,NaN,NaN,NaN,NaN,2.0,26.0,1.0,Alicia,San Román,Gaudul,STL-SAT-0116-27444,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0,0,0,0,0,1,2.0,NaN,10.0,10.0,10.0,1,10.0,19.6,MONO,NaN,10.0,10.0,M,2.0,NO,NaN,NaN,661491417,NaN,01.Muy baja,NaN,NaN,NaN,NaN,NaN,01.Enero,1T-2016,2016,0.419021,2016,1,2016-01
3,2016-01-21 00:00:00,NaN,ASNOR 3,113.0,Baleares,10.0,10.0,Promotor,2,NaN,NaN,NaN,NaN,6,1.0,1,8.0,10.0,10.0,10.0,2.0,30.0,1.0,Lidia,Mra,Gelabert,STL-SAT-0116-26444,1,MAPFRE FAMILIAR,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1,0,0,0,0,1,2,1.0,10.0,10.0,10.0,10.0,1,10.0,85.5,MULTI,NaN,10.0,10.0,M,1.0,NO,10.0,NaN,605728646,NaN,04.Alta,NaN,NaN,NaN,NaN,NaN,01.Enero,1T-2016,2016,0.523919,2016,1,2016-01
4,2016-01-26 00:00:00,NaN,ASGECA 2,85.0,Lleida,6.0,5.0,Detractor,1,10.0,6.0,6.0,6.0,0,1.0,2,NaN,NaN,NaN,NaN,2.0,25.0,1.0,Diana Carolina,Fonseca,Marín,STL-SAT-0116-27071,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0,0,1,0,0,1,1.0,5.0,7.0,7.0,5.0,2,NaN,45.0,MONO,NaN,5.0,5.0,M,NaN,NO,NaN,NaN,640611229,NaN,01.Muy baja,NaN,NaN,NaN,NaN,NaN,01.Enero,1T-2016,2016,1.073565,2016,1,2016-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19591,2021-11-16 17:35:40,ASNOR,Alicante,504.0,Alicante/Alacant,6.0,6.0,Detractor,2,NaN,NaN,NaN,NaN,1,1.0,1,8.0,8.0,5.0,8.0,2.0,31.0,2.0,Cristina,López,Pérez,STL-SAT-1121-199614,1,ALLIANZ,7.0,98.0,NO HE TENIDO NINGUN PROBLEMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,1,0,0,0,1,1.0,7.0,6.0,6.0,7.0,1,5.0,NaN,MONO,NaN,5.0,6.0,M,2.0,NO,NaN,NaN,680508499,NaN,01.Muy baja,2.0,NaN,NaN,NaN,NaN,11.Noviembre,4T-2021,2021,1.123121,2021,11,2021-11
19592,2021-11-17 20:49:06,CENTRO,Madrid Ciudad - Norte,131.0,Madrid,10.0,10.0,Promotor,2,NaN,NaN,NaN,NaN,1,1.0,2,NaN,NaN,NaN,NaN,2.0,39.0,2.0,Noelia,Calamonte,Alvarez,STL-SAT-1121-199618,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,0,0,0,2,0,3,2.0,NaN,10.0,10.0,10.0,1,10.0,NaN,MULTI,NaN,10.0,10.0,M,NaN,NO,NaN,NaN,647989187,NaN,01.Muy baja,2.0,NaN,NaN,NaN,NaN,11.Noviembre,4T-2021,2021,0.406643,2021,11,2021-11
19593,2021-11-22 13:10:53,CENTRO,Madrid Ciudad - Sur,173.0,Madrid,9.0,10.0,Promotor,2,NaN,NaN,NaN,NaN,1,1.0,2,NaN,NaN,NaN,NaN,2.0,63.0,4.0,Maria del Rosario,Ase

In [ ]:
#df["Mes"] = df["FECHA_ENCUESTA"].dt.month

#### 3.1 Separarate between categories

In [ ]:
#Numerical variables
variables_numericas = ["Antiguedad","EDAD","N_SINIESTROS","NSIN_AFP","NSIN_DEC","NSIN_HOG","NSIN_SLD","NSIN_VIA","NSIN_VIR","NUM_POL_AFP","NUM_POL_DEC","NUM_POL_HOG","NUM_POL_LSLD","NUM_POL_TOTAL","NUM_POL_VIA","NUM_POL_VIR","PUNTUACION"]

In [ ]:
#categorical cualitative variables
variables_categoricas_cualitativas = ["AREA","PROVINCIA",
                                      "VINCULACION","sexolis","DESC_MODALIDAD","RAMO","SINIESTROS"
,"TRIM"]
#categorical cuantitative variables
variables_categoricas_cuantitativas = ["AGENCIA",
                                       "CUANTIA","MODALIDAD","RAMO1","RECO"]

In [ ]:
#Separamos los datos
df_inicial_variables_numericas = df[variables_numericas]
df_inicial_variables_categoricas_cualitativas = df[variables_categoricas_cualitativas]
df_inicial_variables_categoricas_cuantitativas = df[variables_categoricas_cuantitativas]

In [ ]:
df_inicial_variables_categoricas_cuantitativas.dtypes

AGENCIA      float64
CUANTIA      float64
MODALIDAD    float64
RAMO1        float64
RECO         float64
dtype: object

#### 3.2 Imputing values numerical variables

###### 3.21 Imputing Empty Values

After removing all the empty values the **data-set** is considerably reduced

In [ ]:
print("Number of empty rows in the numerical values is",df_inicial_variables_numericas.isnull().sum().sum())

Numero de filas vacias en variables numericas 22015


In [ ]:
#Observe number of empty values per column
df_inicial_variables_numericas.isnull().sum()

Antiguedad          0
EDAD                0
N_SINIESTROS        0
NSIN_AFP         3145
NSIN_DEC         3145
NSIN_HOG         3145
NSIN_SLD         3145
NSIN_VIA         3145
NSIN_VIR         3145
NUM_POL_AFP         0
NUM_POL_DEC         0
NUM_POL_HOG         0
NUM_POL_LSLD        0
NUM_POL_TOTAL       0
NUM_POL_VIA         0
NUM_POL_VIR         0
PUNTUACION       3145
dtype: int64

Vemos un diagrama las variables vacias, los numeros en la parte superior nos indican cuantas variables completas hay.

We are going to create a diagram of all the **empty values**, the numbers at the top of the diagram indicates as the number of complete **observations** there are.

In [ ]:
msno.bar(df_inicial_variables_numericas)

NameError: ignored

In [ ]:
#Eliminate empty values
df_eliminacion_vacios_variables_numericas = df_inicial_variables_numericas.dropna()

In [ ]:
#Observe dataframe after removing empty values
df_eliminacion_vacios_variables_numericas.head()

,Antiguedad,EDAD,N_SINIESTROS,NSIN_AFP,NSIN_DEC,NSIN_HOG,NSIN_SLD,NSIN_VIA,NSIN_VIR,NUM_POL_AFP,NUM_POL_DEC,NUM_POL_HOG,NUM_POL_LSLD,NUM_POL_TOTAL,NUM_POL_VIA,NUM_POL_VIR,PUNTUACION
0,1,26,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,2,0,0,54.6
1,1,27,3,0.0,0.0,0.0,3.0,0.0,0.0,0,0,0,1,1,0,0,73.5
2,5,26,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,19.6
3,6,30,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,2,0,0,85.5
4,0,25,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,45.0


After removing all the empty values the **data-set** is considerably reduced

###### 3.22 Input Mean

Instead of removing empty values we can input the mean of the complete observations and fill the empty values with them.

In [ ]:
#Viewing numerical values
df_inicial_variables_numericas

,Antiguedad,EDAD,N_SINIESTROS,NSIN_AFP,NSIN_DEC,NSIN_HOG,NSIN_SLD,NSIN_VIA,NSIN_VIR,NUM_POL_AFP,NUM_POL_DEC,NUM_POL_HOG,NUM_POL_LSLD,NUM_POL_TOTAL,NUM_POL_VIA,NUM_POL_VIR,PUNTUACION
0,1,26,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,2,0,0,54.6
1,1,27,3,0.0,0.0,0.0,3.0,0.0,0.0,0,0,0,1,1,0,0,73.5
2,5,26,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,19.6
3,6,30,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,2,0,0,85.5
4,0,25,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19591,1,31,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,1,0,0,NaN
19592,1,39,0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,3,0,2,NaN
19593,1,63,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,1,0,0,NaN
19594,12,41,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,2,0,1,NaN


In [ ]:
#Input the mean
df_inicial_variables_numericas_imputacion_media = df_inicial_variables_numericas.fillna(df_inicial_variables_numericas.mean())

In [ ]:
#Observe results
df_inicial_variables_numericas_imputacion_media.head(5)

,Antiguedad,EDAD,N_SINIESTROS,NSIN_AFP,NSIN_DEC,NSIN_HOG,NSIN_SLD,NSIN_VIA,NSIN_VIR,NUM_POL_AFP,NUM_POL_DEC,NUM_POL_HOG,NUM_POL_LSLD,NUM_POL_TOTAL,NUM_POL_VIA,NUM_POL_VIR,PUNTUACION
0,1,26,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,2,0,0,54.6
1,1,27,3,0.0,0.0,0.0,3.0,0.0,0.0,0,0,0,1,1,0,0,73.5
2,5,26,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,19.6
3,6,30,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,2,0,0,85.5
4,0,25,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,45.0


In [ ]:
#Observe diagram
msno.bar(df_inicial_variables_numericas_imputacion_media)

##### 3.23 Input Median

We apply the input to empty values using the median of the values

In [ ]:
#Input using median variables
df_inicial_variables_numericas_imputacion_mediana = df_inicial_variables_numericas.fillna(df_inicial_variables_numericas.median())

In [ ]:
#Observe first five observations
df_inicial_variables_numericas_imputacion_mediana.head(5)

,Antiguedad,EDAD,N_SINIESTROS,NSIN_AFP,NSIN_DEC,NSIN_HOG,NSIN_SLD,NSIN_VIA,NSIN_VIR,NUM_POL_AFP,NUM_POL_DEC,NUM_POL_HOG,NUM_POL_LSLD,NUM_POL_TOTAL,NUM_POL_VIA,NUM_POL_VIR,PUNTUACION
0,1,26,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,2,0,0,54.6
1,1,27,3,0.0,0.0,0.0,3.0,0.0,0.0,0,0,0,1,1,0,0,73.5
2,5,26,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,19.6
3,6,30,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,2,0,0,85.5
4,0,25,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,45.0


In [ ]:
#Plot diagram
msno.bar(df_inicial_variables_numericas_imputacion_media)

###### 3.24 KNN Imputer

KNN is the K-nearest neighbors algorithm, it can be used to find similarities between the data and input the missing data

In [ ]:
#Select neighbors=2 and input KNNImputer
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
df_inicial_variables_numericas_imputacion_knn= imputer.fit_transform(df_inicial_variables_numericas)

In [ ]:
#Create Dataframe
df_inicial_variables_numericas_imputacion_knn = pd.DataFrame(df_inicial_variables_numericas_imputacion_knn,columns=df_inicial_variables_numericas.columns)

In [ ]:
df_inicial_variables_numericas_imputacion_knn

,Antiguedad,EDAD,N_SINIESTROS,NSIN_AFP,NSIN_DEC,NSIN_HOG,NSIN_SLD,NSIN_VIA,NSIN_VIR,NUM_POL_AFP,NUM_POL_DEC,NUM_POL_HOG,NUM_POL_LSLD,NUM_POL_TOTAL,NUM_POL_VIA,NUM_POL_VIR,PUNTUACION
0,1.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,54.60
1,1.0,27.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,73.50
2,5.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19.60
3,6.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,85.50
4,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,45.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19591,1.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,72.45
19592,1.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,2.0,45.00
19593,1.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,9.00
19594,12.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,129.50


#### 3.3 Imputing values for categorical variables

We repeat the same process for the input of data, but as there are cualitative variables we need to convert them to numerical. For that we use  **one-hot encoding** in order to encode de data into binary codification.


##### 3.1 One-hot encoding valores categoricas cualitativas

In [ ]:
#Convert the data into one-hot encoding
df_one_hot_variables_categoricas_cualitativas = df_inicial_variables_categoricas_cualitativas
for column in df_inicial_variables_categoricas_cualitativas:
    dummy = pd.get_dummies(df_inicial_variables_categoricas_cualitativas[column],prefix=column)
    df_one_hot_variables_categoricas_cualitativas = df_one_hot_variables_categoricas_cualitativas.drop(column,axis=1)
    df_one_hot_variables_categoricas_cualitativas = pd.concat([df_one_hot_variables_categoricas_cualitativas,dummy],axis=1)

In [ ]:
#Number of columns given one-hot encoding 
print("Number of columns using one-hot encoding",len(df_one_hot_variables_categoricas_cualitativas.columns))

Numero de columnas usando one-hot encoding 235


##### 3.2 Label-Encoding  cualitative variables

In [ ]:
#Label Encoder
df_label_encoding_variables_categoricas_cualitativas=df_inicial_variables_categoricas_cualitativas.apply(preprocessing.LabelEncoder().fit_transform)

In [ ]:
# Info 
df_label_encoding_variables_categoricas_cualitativas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19596 entries, 0 to 19595
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   AREA            19596 non-null  int64
 1   PROVINCIA       19596 non-null  int64
 2   VINCULACION     19596 non-null  int64
 3   sexolis         19596 non-null  int64
 4   DESC_MODALIDAD  19596 non-null  int64
 5   RAMO            19596 non-null  int64
 6   SINIESTROS      19596 non-null  int64
 7   TRIM            19596 non-null  int64
dtypes: int64(8)
memory usage: 1.2 MB


In [ ]:
df_label_encoding_variables_categoricas_cualitativas

,AREA,PROVINCIA,VINCULACION,sexolis,DESC_MODALIDAD,RAMO,SINIESTROS,TRIM
0,32,35,1,1,65,1,0,0
1,2,38,2,1,65,0,1,0
2,29,35,0,1,65,0,0,0
3,10,10,3,1,65,1,0,0
4,7,33,0,1,65,0,0,0
...,...,...,...,...,...,...,...,...
19591,18,4,0,1,56,0,0,23
19592,53,35,0,1,60,1,0,23
19593,54,35,0,1,52,0,0,23
19594,19,25,3,1,60,1,0,23


In [ ]:
print("Number of empty rows in numerical variables",df_label_encoding_variables_categoricas_cualitativas.isnull().sum().sum())

Numero de filas vacias en variables numericas 0


In [ ]:
#Number of empty values for cualitative variables
df_label_encoding_variables_categoricas_cualitativas.isnull().sum()

AREA              0
PROVINCIA         0
VINCULACION       0
sexolis           0
DESC_MODALIDAD    0
RAMO              0
SINIESTROS        0
TRIM              0
dtype: int64

In [ ]:
#Graph of empty values
msno.bar(df_label_encoding_variables_categoricas_cualitativas)

#### 3.4 Imputing values for cuantivative categorical variables

In [ ]:
#### 

In [ ]:
print("Numero de filas vacias en variables numericas",df_inicial_variables_categoricas_cuantitativas.isnull().sum().sum())

Numero de filas vacias en variables numericas 19420


In [ ]:
df_inicial_variables_categoricas_cuantitativas.isnull().sum()

AGENCIA      6041
CUANTIA         0
MODALIDAD    4408
RAMO1        7553
RECO         1418
dtype: int64

##### 3.32 KNN Imputer

In [ ]:
df_inicial_variables_categoricas_cuantitativas

,AGENCIA,CUANTIA,MODALIDAD,RAMO1,RECO
0,2.0,0.00,NaN,NaN,10.0
1,536.0,64.00,NaN,NaN,10.0
2,172.0,0.00,NaN,NaN,10.0
3,113.0,0.00,NaN,NaN,10.0
4,85.0,0.00,NaN,NaN,5.0
...,...,...,...,...,...
19591,504.0,0.00,610.0,NaN,5.0
19592,131.0,0.00,463.0,NaN,10.0
19593,173.0,0.00,4001.0,NaN,10.0
19594,65.0,0.00,463.0,NaN,5.0


In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
df_inicial_variables_categoricas_cuantitativas_knn= imputer.fit_transform(df_inicial_variables_categoricas_cuantitativas)

In [ ]:
df_inicial_variables_categoricas_cuantitativas_knn = pd.DataFrame(df_inicial_variables_categoricas_cuantitativas_knn,columns=df_inicial_variables_categoricas_cuantitativas.columns)

In [ ]:
df_inicial_variables_categoricas_cuantitativas_knn

,AGENCIA,CUANTIA,MODALIDAD,RAMO1,RECO
0,2.0,0.00,534.0,234.5,10.0
1,536.0,64.00,2201.0,22.0,10.0
2,172.0,0.00,602.0,6.0,10.0
3,113.0,0.00,851.5,8.5,10.0
4,85.0,0.00,538.0,240.0,5.0
...,...,...,...,...,...
19591,504.0,0.00,610.0,6.0,5.0
19592,131.0,0.00,463.0,463.0,10.0
19593,173.0,0.00,4001.0,4001.0,10.0
19594,65.0,0.00,463.0,463.0,5.0


#### 4.1 Removing outliers for categorical cuantitative values

We are going to remove the outliers for categorical cuantitative values using, the **IQR method**

In [ ]:
df_inicial_variables_categoricas_cuantitativas_knn.describe()

,AGENCIA,CUANTIA,MODALIDAD,RAMO1,RECO
count,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000
mean,318.147198,45.457720,1421.530567,297.122244,7.505282
std,261.062133,967.122637,1090.239839,772.175242,2.882422
min,1.000000,0.000000,401.000000,6.000000,0.000000
25%,87.000000,0.000000,602.000000,11.000000,6.000000
50%,172.000000,0.000000,1101.000000,22.000000,8.000000
75%,625.000000,0.000000,2057.750000,253.500000,10.000000
max,770.000000,76743.320000,4002.000000,4002.000000,10.000000


In [ ]:

Q1 = df_inicial_variables_categoricas_cuantitativas_knn.quantile(0.25)
Q3 = df_inicial_variables_categoricas_cuantitativas_knn.quantile(0.75)
IQR = Q3-Q1

In [ ]:
Q1

AGENCIA       87.0
CUANTIA        0.0
MODALIDAD    602.0
RAMO1         11.0
RECO           6.0
Name: 0.25, dtype: float64

In [ ]:
IQR

AGENCIA       538.00
CUANTIA         0.00
MODALIDAD    1455.75
RAMO1         242.50
RECO            4.00
dtype: float64

In [ ]:
out_param = 3

In [ ]:
df_inicial_variables_categoricas_cuantitativas_knn_outliers = df_inicial_variables_categoricas_cuantitativas_knn[~((df_inicial_variables_categoricas_cuantitativas_knn < (Q1 - out_param * IQR)) |(df_inicial_variables_categoricas_cuantitativas_knn > (Q3 + out_param * IQR))).any(axis=1)]


#### 4.2 Removing outliers for categorical cualitative values

We once again apply the same process to remove outliers

In [ ]:
df_label_encoding_variables_categoricas_cualitativas.describe()

,AREA,PROVINCIA,VINCULACION,sexolis,DESC_MODALIDAD,RAMO,SINIESTROS,TRIM
count,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000
mean,29.848643,27.173709,1.715860,1.473209,30.925648,0.412431,0.093437,11.335885
std,22.046615,15.604157,1.453575,0.513106,24.313153,0.492285,0.291052,6.828102
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.000000,11.000000,0.000000,1.000000,11.000000,0.000000,0.000000,5.000000
50%,29.000000,31.000000,2.000000,1.000000,18.000000,0.000000,0.000000,11.000000
75%,48.000000,38.000000,3.000000,2.000000,60.000000,1.000000,0.000000,17.000000
max,76.000000,56.000000,4.000000,2.000000,65.000000,1.000000,1.000000,23.000000


In [ ]:

Q1 = df_one_hot_variables_categoricas_cualitativas.quantile(0.25)
Q3 = df_label_encoding_variables_categoricas_cualitativas.quantile(0.75)
IQR = Q3-Q1

In [ ]:
Q1

AREA_AGENTE                0.0
AREA_AGENTES EXCLUSIVOS    0.0
AREA_AMSUR 1               0.0
AREA_AMSUR 2               0.0
AREA_AMSUR 3               0.0
                          ... 
TRIM_4T-2017               0.0
TRIM_4T-2018               0.0
TRIM_4T-2019               0.0
TRIM_4T-2020               0.0
TRIM_4T-2021               0.0
Name: 0.25, Length: 235, dtype: float64

In [ ]:
IQR

AREA                      NaN
AREA_AGENTE               NaN
AREA_AGENTES EXCLUSIVOS   NaN
AREA_AMSUR 1              NaN
AREA_AMSUR 2              NaN
                           ..
VINCULACION_05.Muy alta   NaN
sexolis                   NaN
sexolis_9                 NaN
sexolis_M                 NaN
sexolis_V                 NaN
Length: 243, dtype: float64

In [ ]:
out_param = 3

In [ ]:
df_label_encoding_variables_categoricas_cualitativas_outliers = df_label_encoding_variables_categoricas_cualitativas[~((df_label_encoding_variables_categoricas_cualitativas < (Q1 - out_param * IQR)) |(df_label_encoding_variables_categoricas_cualitativas > (Q3 + out_param * IQR))).any(axis=1)]


In [ ]:
df_label_encoding_variables_categoricas_cualitativas

,AREA,PROVINCIA,VINCULACION,sexolis,DESC_MODALIDAD,RAMO,SINIESTROS,TRIM
0,32,35,1,1,65,1,0,0
1,2,38,2,1,65,0,1,0
2,29,35,0,1,65,0,0,0
3,10,10,3,1,65,1,0,0
4,7,33,0,1,65,0,0,0
...,...,...,...,...,...,...,...,...
19591,18,4,0,1,56,0,0,23
19592,53,35,0,1,60,1,0,23
19593,54,35,0,1,52,0,0,23
19594,19,25,3,1,60,1,0,23


In [ ]:
df_label_encoding_variables_categoricas_cualitativas.boxplot()

####4.3 Removing outliers for numerical values

In [ ]:
df_inicial_variables_numericas_imputacion_knn.describe()

,Antiguedad,EDAD,N_SINIESTROS,NSIN_AFP,NSIN_DEC,NSIN_HOG,NSIN_SLD,NSIN_VIA,NSIN_VIR,NUM_POL_AFP,NUM_POL_DEC,NUM_POL_HOG,NUM_POL_LSLD,NUM_POL_TOTAL,NUM_POL_VIA,NUM_POL_VIR,PUNTUACION
count,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000,19596.000000
mean,19.438916,55.903041,0.293887,0.000689,0.002169,0.081853,0.190396,0.003572,0.000255,0.287457,0.445958,0.438814,0.092162,1.723056,0.220555,0.203409,76.304173
std,16.078640,12.674712,2.519154,0.025499,0.045830,0.341088,2.444147,0.062380,0.029453,0.456964,0.503409,0.639023,0.300680,1.023132,0.545400,0.510028,50.342891
min,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,-58.500000
25%,6.000000,46.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,44.037500
50%,16.000000,56.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,63.625000
75%,29.000000,66.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,2.000000,0.000000,0.000000,98.400000
max,86.000000,80.000000,139.000000,1.000000,1.000000,5.000000,139.000000,2.000000,4.000000,3.000000,3.000000,10.000000,4.000000,12.000000,10.000000,4.000000,620.250000


In [ ]:

Q1 = df_inicial_variables_numericas_imputacion_knn.quantile(0.25)
Q3 = df_inicial_variables_numericas_imputacion_knn.quantile(0.75)
IQR = Q3-Q1

In [ ]:
Q1

Antiguedad        6.0000
EDAD             46.0000
N_SINIESTROS      0.0000
NSIN_AFP          0.0000
NSIN_DEC          0.0000
NSIN_HOG          0.0000
NSIN_SLD          0.0000
NSIN_VIA          0.0000
NSIN_VIR          0.0000
NUM_POL_AFP       0.0000
NUM_POL_DEC       0.0000
NUM_POL_HOG       0.0000
NUM_POL_LSLD      0.0000
NUM_POL_TOTAL     1.0000
NUM_POL_VIA       0.0000
NUM_POL_VIR       0.0000
PUNTUACION       44.0375
Name: 0.25, dtype: float64

In [ ]:
IQR

Antiguedad       23.0000
EDAD             20.0000
N_SINIESTROS      0.0000
NSIN_AFP          0.0000
NSIN_DEC          0.0000
NSIN_HOG          0.0000
NSIN_SLD          0.0000
NSIN_VIA          0.0000
NSIN_VIR          0.0000
NUM_POL_AFP       1.0000
NUM_POL_DEC       1.0000
NUM_POL_HOG       1.0000
NUM_POL_LSLD      0.0000
NUM_POL_TOTAL     1.0000
NUM_POL_VIA       0.0000
NUM_POL_VIR       0.0000
PUNTUACION       54.3625
dtype: float64

In [ ]:
out_param = 10

In [ ]:
df_inicial_variables_numericas_imputacion_knn_outliers = df_inicial_variables_numericas_imputacion_knn[~((df_inicial_variables_numericas_imputacion_knn < (Q1 - out_param * IQR)) |(df_inicial_variables_numericas_imputacion_knn > (Q3 + out_param * IQR))).any(axis=1)]


In [ ]:
df_inicial_variables_numericas_imputacion_knn_outliers

,Antiguedad,EDAD,N_SINIESTROS,NSIN_AFP,NSIN_DEC,NSIN_HOG,NSIN_SLD,NSIN_VIA,NSIN_VIR,NUM_POL_AFP,NUM_POL_DEC,NUM_POL_HOG,NUM_POL_LSLD,NUM_POL_TOTAL,NUM_POL_VIA,NUM_POL_VIR,PUNTUACION
0,1.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,54.60
2,5.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19.60
9,3.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,56.00
10,7.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,32.00
11,5.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,60.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19572,8.0,74.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,32.00
19574,6.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,83.75
19575,6.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,50.00
19589,2.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,42.00


In [ ]:
#df_inicial_variables_numericas_imputacion_mediana_outliers.boxplot()

#### 5.1 Preprocessing of numerical values

We apply a **Min-Max scaler** in order to normalize the data

In [ ]:
#Scalate Data
scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(df_inicial_variables_numericas_imputacion_knn_outliers)

df_inicial_variables_numericas_imputacion_knn_outliers_preprocesado= pd.DataFrame(X,columns=df_inicial_variables_numericas_imputacion_knn_outliers.columns,index=df_inicial_variables_numericas_imputacion_knn_outliers.index)


NameError: ignored

In [ ]:
df_inicial_variables_numericas_imputacion_knn_outliers

,Antiguedad,EDAD,N_SINIESTROS,NSIN_AFP,NSIN_DEC,NSIN_HOG,NSIN_SLD,NSIN_VIA,NSIN_VIR,NUM_POL_AFP,NUM_POL_DEC,NUM_POL_HOG,NUM_POL_LSLD,NUM_POL_TOTAL,NUM_POL_VIA,NUM_POL_VIR,PUNTUACION
0,1.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,54.60
2,5.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19.60
9,3.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,56.00
10,7.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,32.00
11,5.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,60.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19572,8.0,74.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,32.00
19574,6.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,83.75
19575,6.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,50.00
19589,2.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,42.00


### 4.0 Generating X and Y

Here we will generate the labels in order to get X and Y. We join the datasets in order to get X, and get NPS Y. 

En este caso solo para dar un ejemplo elegimos solo las variables **Antiguedad** y **Edad**

#### 4.1 Generate X values

We are going to generate the **dependent variables** in order to train the models. We have generated three independent **dataframes** we now have to concatenate them.

In [ ]:
#The categorical cuantitative imputed using knn outliers variables
df_inicial_variables_categoricas_cuantitativas_knn_outliers

,AGENCIA,CUANTIA,MODALIDAD,RAMO1,RECO
0,2.0,0.0,534.0,234.5,10.0
2,172.0,0.0,602.0,6.0,10.0
3,113.0,0.0,851.5,8.5,10.0
4,85.0,0.0,538.0,240.0,5.0
6,67.0,0.0,2201.5,22.0,10.0
...,...,...,...,...,...
19589,741.0,0.0,3301.0,33.0,10.0
19590,46.0,0.0,3508.0,35.0,10.0
19591,504.0,0.0,610.0,6.0,5.0
19592,131.0,0.0,463.0,463.0,10.0


In [ ]:
#The numerical knn outliers variables
df_inicial_variables_numericas_imputacion_knn_outliers

,Antiguedad,EDAD,N_SINIESTROS,NSIN_AFP,NSIN_DEC,NSIN_HOG,NSIN_SLD,NSIN_VIA,NSIN_VIR,NUM_POL_AFP,NUM_POL_DEC,NUM_POL_HOG,NUM_POL_LSLD,NUM_POL_TOTAL,NUM_POL_VIA,NUM_POL_VIR,PUNTUACION
0,1.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,54.60
2,5.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19.60
9,3.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,56.00
10,7.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,32.00
11,5.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,60.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19572,8.0,74.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,32.00
19574,6.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,83.75
19575,6.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,50.00
19589,2.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,42.00


In [ ]:
#The cualitative categorical variables 
df_label_encoding_variables_categoricas_cualitativas

,AREA,PROVINCIA,VINCULACION,sexolis,DESC_MODALIDAD,RAMO,SINIESTROS,TRIM
0,32,35,1,1,65,1,0,0
1,2,38,2,1,65,0,1,0
2,29,35,0,1,65,0,0,0
3,10,10,3,1,65,1,0,0
4,7,33,0,1,65,0,0,0
...,...,...,...,...,...,...,...,...
19591,18,4,0,1,56,0,0,23
19592,53,35,0,1,60,1,0,23
19593,54,35,0,1,52,0,0,23
19594,19,25,3,1,60,1,0,23


In [ ]:
df = d2
#Place the three initial dataframes into a list
frames = [df_inicial_variables_categoricas_cuantitativas_knn_outliers,
          df_inicial_variables_numericas_imputacion_knn_outliers,
          df_label_encoding_variables_categoricas_cualitativas]
#Concat this list
df_final_df = pd.concat(frames,axis=1)

In [ ]:
#Show final dataframe
df_final_df

,AGENCIA,CUANTIA,MODALIDAD,RAMO1,RECO,Antiguedad,EDAD,N_SINIESTROS,NSIN_AFP,NSIN_DEC,NSIN_HOG,NSIN_SLD,NSIN_VIA,NSIN_VIR,NUM_POL_AFP,NUM_POL_DEC,NUM_POL_HOG,NUM_POL_LSLD,NUM_POL_TOTAL,NUM_POL_VIA,NUM_POL_VIR,PUNTUACION,AREA,PROVINCIA,VINCULACION,sexolis,DESC_MODALIDAD,RAMO,SINIESTROS,TRIM
0,2.0,0.0,534.0,234.5,10.0,1.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,54.60,32,35,1,1,65,1,0,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,38,2,1,65,0,1,0
2,172.0,0.0,602.0,6.0,10.0,5.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19.60,29,35,0,1,65,0,0,0
3,113.0,0.0,851.5,8.5,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,10,3,1,65,1,0,0
4,85.0,0.0,538.0,240.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,33,0,1,65,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19591,504.0,0.0,610.0,6.0,5.0,1.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,72.45,18,4,0,1,56,0,0,23
19592,131.0,0.0,463.0,463.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,35,0,1,60,1,0,23
19593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54,35,0,1,52,0,0,23
19594,65.0,0.0,463.0,463.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,25,3,1,60,1,0,23


In [ ]:
#Transform dataset
df_final= imputer.fit_transform(df_final_df)

In [ ]:
#Function to include shits in order to be able to input the data to a timeseries model 
def include_shifts(df, shifts=[1,2,3]):
  dfs = []
  for shift in shifts:
    print(shift)
    df_shifted = df.shift(shift)
    df_shifted.columns = [f"{col}_{shift}" for col in df_shifted.columns]
    dfs.append(df_shifted)
  totaldf = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), dfs)
  totaldf = totaldf.iloc[max(shifts):]
  return totaldf

In [ ]:
df_final

array([[2.0000e+00, 0.0000e+00, 5.3400e+02, ..., 1.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.3150e+02, 0.0000e+00, 1.7525e+03, ..., 0.0000e+00, 1.0000e+00,
        0.0000e+00],
       [1.7200e+02, 0.0000e+00, 6.0200e+02, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [2.7500e+02, 0.0000e+00, 2.0585e+03, ..., 0.0000e+00, 0.0000e+00,
        2.3000e+01],
       [6.5000e+01, 0.0000e+00, 4.6300e+02, ..., 1.0000e+00, 0.0000e+00,
        2.3000e+01],
       [3.7475e+02, 0.0000e+00, 6.1000e+02, ..., 0.0000e+00, 1.0000e+00,
        2.3000e+01]])

In [ ]:
#Observe columns
df_final_df.columns

In [ ]:
#Transfer columns 
df_final_df_2 = pd.DataFrame(df_final,columns=df_final_df.columns)
df_final = df_final_df_2
#Convert date
df_final["Año-Mes"] = pd.to_datetime(df['FECHA_ENCUESTA']).dt.to_period('M')
#Set date as index
df_final = df_final.set_index("Año-Mes")

Index(['AGENCIA', 'CUANTIA', 'MODALIDAD', 'RAMO1', 'RECO', 'Antiguedad',
       'EDAD', 'N_SINIESTROS', 'NSIN_AFP', 'NSIN_DEC', 'NSIN_HOG', 'NSIN_SLD',
       'NSIN_VIA', 'NSIN_VIR', 'NUM_POL_AFP', 'NUM_POL_DEC', 'NUM_POL_HOG',
       'NUM_POL_LSLD', 'NUM_POL_TOTAL', 'NUM_POL_VIA', 'NUM_POL_VIR',
       'PUNTUACION', 'AREA', 'PROVINCIA', 'VINCULACION', 'sexolis',
       'DESC_MODALIDAD', 'RAMO', 'SINIESTROS', 'TRIM'],
      dtype='object')

In [ ]:
#Observe final dataframe
df_final

,AGENCIA,CUANTIA,MODALIDAD,RAMO1,RECO,Antiguedad,EDAD,N_SINIESTROS,NSIN_AFP,NSIN_DEC,NSIN_HOG,NSIN_SLD,NSIN_VIA,NSIN_VIR,NUM_POL_AFP,NUM_POL_DEC,NUM_POL_HOG,NUM_POL_LSLD,NUM_POL_TOTAL,NUM_POL_VIA,NUM_POL_VIR,PUNTUACION,AREA,PROVINCIA,VINCULACION,sexolis,DESC_MODALIDAD,RAMO,SINIESTROS,TRIM
Año-Mes,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01,2.00,0.0,534.0,234.5,10.0,1.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,54.600,32.0,35.0,1.0,1.0,65.0,1.0,0.0,0.0
2016-01,331.50,0.0,1752.5,17.5,9.0,3.0,43.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,70.250,2.0,38.0,2.0,1.0,65.0,0.0,1.0,0.0
2016-01,172.00,0.0,602.0,6.0,10.0,5.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19.600,29.0,35.0,0.0,1.0,65.0,0.0,0.0,0.0
2016-01,113.00,0.0,851.5,8.5,10.0,20.5,49.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.5,0.0,1.5,0.0,0.0,84.300,10.0,10.0,3.0,1.0,65.0,1.0,0.0,0.0
2016-01,85.00,0.0,538.0,240.0,5.0,43.0,71.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,43.200,7.0,33.0,0.0,1.0,65.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11,504.00,0.0,610.0,6.0,5.0,1.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,72.450,18.0,4.0,0.0,1.0,56.0,0.0,0.0,23.0
2021-11,131.00,0.0,463.0,463.0,10.0,11.0,48.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,1.0,0.0,0.0,55.750,53.0,35.0,0.0,1.0,60.0,1.0,0.0,23.0
2021-11,275.00,0.0,2058.5,20.5,7.0,3.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,98.250,54.0,35.0,0.0,1.0,52.0,0.0,0.0,23.0


In [ ]:
#Check number 
df_final.isnull().sum()

AGENCIA           0
CUANTIA           0
MODALIDAD         0
RAMO1             0
RECO              0
Antiguedad        0
EDAD              0
N_SINIESTROS      0
NSIN_AFP          0
NSIN_DEC          0
NSIN_HOG          0
NSIN_SLD          0
NSIN_VIA          0
NSIN_VIR          0
NUM_POL_AFP       0
NUM_POL_DEC       0
NUM_POL_HOG       0
NUM_POL_LSLD      0
NUM_POL_TOTAL     0
NUM_POL_VIA       0
NUM_POL_VIR       0
PUNTUACION        0
AREA              0
PROVINCIA         0
VINCULACION       0
sexolis           0
DESC_MODALIDAD    0
RAMO              0
SINIESTROS        0
TRIM              0
dtype: int64

In [ ]:
#Get X values by getting the medain
X = df_final.groupby(pd.Grouper(freq="M"))[df_final.columns].median()
# Include shits
X = include_shifts(X)

In [ ]:
#Observe X
X

,AGENCIA_1,CUANTIA_1,MODALIDAD_1,RAMO1_1,RECO_1,Antiguedad_1,EDAD_1,N_SINIESTROS_1,NSIN_AFP_1,NSIN_DEC_1,NSIN_HOG_1,NSIN_SLD_1,NSIN_VIA_1,NSIN_VIR_1,NUM_POL_AFP_1,NUM_POL_DEC_1,NUM_POL_HOG_1,NUM_POL_LSLD_1,NUM_POL_TOTAL_1,NUM_POL_VIA_1,NUM_POL_VIR_1,PUNTUACION_1,AREA_1,PROVINCIA_1,VINCULACION_1,sexolis_1,DESC_MODALIDAD_1,RAMO_1,SINIESTROS_1,TRIM_1,AGENCIA_2,CUANTIA_2,MODALIDAD_2,RAMO1_2,RECO_2,Antiguedad_2,EDAD_2,N_SINIESTROS_2,NSIN_AFP_2,NSIN_DEC_2,...,NUM_POL_VIR_2,PUNTUACION_2,AREA_2,PROVINCIA_2,VINCULACION_2,sexolis_2,DESC_MODALIDAD_2,RAMO_2,SINIESTROS_2,TRIM_2,AGENCIA_3,CUANTIA_3,MODALIDAD_3,RAMO1_3,RECO_3,Antiguedad_3,EDAD_3,N_SINIESTROS_3,NSIN_AFP_3,NSIN_DEC_3,NSIN_HOG_3,NSIN_SLD_3,NSIN_VIA_3,NSIN_VIR_3,NUM_POL_AFP_3,NUM_POL_DEC_3,NUM_POL_HOG_3,NUM_POL_LSLD_3,NUM_POL_TOTAL_3,NUM_POL_VIA_3,NUM_POL_VIR_3,PUNTUACION_3,AREA_3,PROVINCIA_3,VINCULACION_3,sexolis_3,DESC_MODALIDAD_3,RAMO_3,SINIESTROS_3,TRIM_3
Año-Mes,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-04,169.00,0.0,1101.0,33.0,8.0,18.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,1.0,0.0,0.0,62.40,11.0,31.0,2.0,2.0,65.0,0.0,0.0,0.0,168.00,0.0,1101.0,22.5,8.0,17.0,58.0,0.0,0.0,0.0,...,0.0,62.4000,11.0,30.0,1.0,1.0,65.0,0.0,0.0,0.0,165.00,0.0,1101.0,22.0,8.0,15.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,1.0,0.0,0.0,56.6000,11.0,32.0,1.0,1.0,65.0,0.0,0.0,0.0
2016-05,173.00,0.0,1101.0,22.0,8.0,17.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,1.0,0.0,0.0,60.80,10.0,29.0,2.0,1.0,65.0,0.0,0.0,6.0,169.00,0.0,1101.0,33.0,8.0,18.0,59.0,0.0,0.0,0.0,...,0.0,62.4000,11.0,31.0,2.0,2.0,65.0,0.0,0.0,0.0,168.00,0.0,1101.0,22.5,8.0,17.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,1.0,0.0,0.0,62.4000,11.0,30.0,1.0,1.0,65.0,0.0,0.0,0.0
2016-06,162.00,0.0,1101.0,25.0,8.0,17.0,57.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,1.0,0.0,0.0,60.80,11.0,32.0,1.0,1.0,65.0,0.0,0.0,6.0,173.00,0.0,1101.0,22.0,8.0,17.0,57.0,0.0,0.0,0.0,...,0.0,60.8000,10.0,29.0,2.0,1.0,65.0,0.0,0.0,6.0,169.00,0.0,1101.0,33.0,8.0,18.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,1.0,0.0,0.0,62.4000,11.0,31.0,2.0,2.0,65.0,0.0,0.0,0.0
2016-07,173.00,0.0,1101.0,22.0,8.0,18.5,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.5,0.0,1.0,0.0,0.0,62.40,11.0,35.0,1.0,1.0,65.0,0.0,0.0,6.0,162.00,0.0,1101.0,25.0,8.0,17.0,57.5,0.0,0.0,0.0,...,0.0,60.8000,11.0,32.0,1.0,1.0,65.0,0.0,0.0,6.0,173.00,0.0,1101.0,22.0,8.0,17.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,1.0,0.0,0.0,60.8000,10.0,29.0,2.0,1.0,65.0,0.0,0.0,6.0
2016-08,178.00,0.0,1243.5,29.0,8.0,18.0,59.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,1.0,0.0,0.0,60.00,11.0,30.0,2.0,1.0,65.0,0.0,0.0,12.0,173.00,0.0,1101.0,22.0,8.0,18.5,58.0,0.0,0.0,0.0,...,0.0,62.4000,11.0,35.0,1.0,1.0,65.0,0.0,0.0,6.0,162.00,0.0,1101.0,25.0,8.0,17.0,57.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,1.0,0.0,0.0,60.8000,11.0,32.0,1.0,1.0,65.0,0.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07,360.50,0.0,1101.0,20.5,9.0,14.5,58.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,1.0,0.0,0.0,60.00,55.0,35.0,2.0,1.0,13.0,0.0,0.0,11.0,374.75,0.0,1101.0,11.0,8.0,18.0,59.0,0.0,0.0,0.0,...,0.0,58.8375,48.0,31.0,1.0,1.0,12.0,0.0,0.0,11.0,360.50,0.0,1101.0,11.0,8.0,18.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,1.0,0.0,0.0,59.1250,48.0,31.0,1.0,1.0,11.0,0.0,0.0,11.0
2021-08,374.75,0.0,1101.0,11.0,8.0,19.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,1.0,0.0,0.0,65.60,51.0,32.0,2.0,1.0,13.0,0.0,0.0,17.0,360.50,0.0,1101.0,20.5,9.0,14.5,58.5,0.0,0.0,0.0,...,0.0,60.0000,55.0,35.0,2.0,1.0,13.0,0.0,0.0,11.0,374.75,0.0,1101.0,11.0,8.0,18.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,1.0,0.0,0.0,58.8375,48.0,31.0,1.0,1.0,12.0,0.0,0.0,11.0
2021-09,3

#### 4.2 Add NPS

The **NPS** data comes from another dataset, we loaded in a dataframe named nps_df, we need to join this with the **initial dataset**

In [ ]:
#We can observe the nps df
nps_df

,Etiquetas de fila,FECHA_ENCUESTA,Promedio de PROM (PONDERADO),Promedio de NEU (PONDERADO,Promedio de DET (PONDERADO),NPS
0,201601.Enero,2016-01-01,0.488499,0.320136,0.185113,0.303386
1,201602.Febrero,2016-02-01,0.490933,0.321561,0.181199,0.309734
2,201603.Marzo,2016-03-01,0.508864,0.269049,0.225050,0.283814
3,201604.Abril,2016-04-01,0.529178,0.319789,0.152801,0.376378
4,201605.Mayo,2016-05-01,0.438591,0.385417,0.168912,0.269680
...,...,...,...,...,...,...
66,202107.Julio,2021-07-01,0.492255,0.327343,0.173006,0.319249
67,202108.Agosto,2021-08-01,0.613841,0.174724,0.199610,0.414231
68,202109.Septiembre,2021-09-01,0.534032,0.210234,0.254238,0.279794
69,202110.Octubre,2021-10-01,0.548107,0.257555,0.193531,0.354576


In [ ]:
#Convert column to datetime
nps_df["FECHA_ENCUESTA"] = pd.to_datetime(nps_df['FECHA_ENCUESTA']).dt.to_period('M')
#Set index to the date
nps_df = nps_df.set_index("FECHA_ENCUESTA")
nps_df

,Etiquetas de fila,Promedio de PROM (PONDERADO),Promedio de NEU (PONDERADO,Promedio de DET (PONDERADO),NPS
FECHA_ENCUESTA,,,,,
2016-01,201601.Enero,0.488499,0.320136,0.185113,0.303386
2016-02,201602.Febrero,0.490933,0.321561,0.181199,0.309734
2016-03,201603.Marzo,0.508864,0.269049,0.225050,0.283814
2016-04,201604.Abril,0.529178,0.319789,0.152801,0.376378
2016-05,201605.Mayo,0.438591,0.385417,0.168912,0.269680
...,...,...,...,...,...
2021-07,202107.Julio,0.492255,0.327343,0.173006,0.319249
2021-08,202108.Agosto,0.613841,0.174724,0.199610,0.414231
2021-09,202109.Septiembre,0.534032,0.210234,0.254238,0.279794


In [ ]:
#Add new column to the dataframe X
X["NPS_final"] = nps_df["NPS"]
X["NPS_final"] 

### 5.0 PCA

Due to the large number of variables we have we are going to add a PCA (Principal component analysis), in order to reduce the number of variables we have for the independent dataframe

In [ ]:
Y = X["NPS_final"]
X = X.drop("NPS_final",axis=1)
#Apply PCA using a 95% variance
pca = PCA(.95)

In [ ]:
#Apply the PCA
pca.fit(X)
X_transformado = pca.transform(X)
X_transformado.shape

### 6.0 Dividing train and validation test
We are going to divide the data into train and validation sets. We are going to divide the data in different ways in order to check for different time predictions.

### 7.0 14-month division

In this division we are going to divide the data in such a way that the validation data corresponds to one year and therefore the model predicts one single year.

In [ ]:
#Divide data
X_train_anual,X_test_anual,y_train_anual,y_test_anual = train_test_split(X_transformado,Y, test_size=0.2,shuffle=False)

In [ ]:
#Check data we see 14 months of prediction
y_test_anual

Año-Mes
2020-10    0.453546
2020-11    0.383936
2020-12    0.467316
2021-01    0.353062
2021-02    0.275977
2021-03    0.379094
2021-04    0.367181
2021-05    0.287904
2021-06    0.392936
2021-07    0.319249
2021-08    0.414231
2021-09    0.279794
2021-10    0.354576
2021-11    0.238822
Freq: M, Name: NPS_final, dtype: float64

Como se puede observar los datos de test tienen un rango de **Septiembre 2020** a **Octubre 2021**

In [ ]:
#date is the datetime index
date = y_test_anual.index 
#this will return you a numpy array, element is string.
date = date.strftime('%Y-%m-%d') 
 #this will make you numpy array into a list
dstr_anual = date.tolist()

In [ ]:
dstr_anual

['2020-10-31',
 '2020-11-30',
 '2020-12-31',
 '2021-01-31',
 '2021-02-28',
 '2021-03-31',
 '2021-04-30',
 '2021-05-31',
 '2021-06-30',
 '2021-07-31',
 '2021-08-31',
 '2021-09-30',
 '2021-10-31',
 '2021-11-30']

In [ ]:
#We plot our prediction
figure(figsize=(24, 6), dpi=80)
plt.plot(dstr_anual,y_test_anual)
plt.title("Fecha mensual contra el valor del NPS")
plt.xlabel("Fecha")
plt.ylabel("NPS")
plt.show()

### 8.0 3-month division

In [ ]:
from sklearn.model_selection import train_test_split

X_train_trimestral,X_test_trimestral,y_train_trimestral,y_test_trimestral = train_test_split(X,Y, test_size=0.04,shuffle=False)

In [ ]:
y_test_trimestral

Año-Mes
2021-09    0.279794
2021-10    0.354576
2021-11    0.238822
Freq: M, Name: NPS_final, dtype: float64

Como se puede observar los datos de test tienen un rango de **Agosto 2021** a **Octubre 2021**

In [ ]:
date = y_test_trimestral.index #date is the datetime index
date = date.strftime('%Y-%m-%d') #this will return you a numpy array, element is string.
dstr_trimestral = date.tolist() #this will make you numpy array into a list

In [ ]:
dstr_trimestral

['2021-09-30', '2021-10-31', '2021-11-30']

In [ ]:
import seaborn as sns
from matplotlib.pyplot import figure

figure(figsize=(24, 6), dpi=80)
plt.plot(dstr_trimestral,y_test_trimestral)
plt.title("Fecha mensual contra el valor del NPS")
plt.xlabel("Fecha")
plt.ylabel("NPS")
plt.show()

### 9.0 Moving Average model

A moving average is one of the most used algorithms for time-series prediction.

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [ ]:
X_ma = Y.to_frame()

In [ ]:
X_ma = X_ma.append(pd.Series([np.nan]), ignore_index=True)

In [ ]:
X_ma = X_ma.drop(0,axis=1)

In [ ]:
X_ma

,NPS_final
0,0.376378
1,0.269680
2,0.201297
3,0.323791
4,0.357460
...,...
64,0.414231
65,0.279794
66,0.354576
67,0.238822


In [ ]:
# the simple moving average over a period of 30 day
X_ma['SMA_1'] = X_ma.NPS_final.rolling(3, min_periods=2).mean()
X_ma['SMA_3'] = X_ma.NPS_final.rolling(5, min_periods=2).mean()
# the simple moving average over a period of 60 day

X_ma

,NPS_final,SMA_1,SMA_3
0,0.376378,NaN,NaN
1,0.269680,0.323029,0.323029
2,0.201297,0.282451,0.282451
3,0.323791,0.264923,0.292786
4,0.357460,0.294183,0.305721
...,...,...,...
64,0.414231,0.375472,0.356300
65,0.279794,0.337758,0.338823
66,0.354576,0.349533,0.352157
67,0.238822,0.291064,0.321334


In [ ]:
#Observe results
colors = ['gray', 'green', 'orange']

X_ma[["NPS_final","SMA_1","SMA_3"]].plot(color=colors, linewidth=3, figsize=(12,6), alpha=0.8)


plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(labels=['Cambio NPS', "Predicción 1 mes SMA", "Predicción trimestral SMA"], fontsize=14)

plt.title("Cambio NPS", fontsize=20)
plt.xlabel('Fecha', fontsize=16)
plt.ylabel('Cambio NPS')
plt.savefig("MA.png")

We apply a simple moving averages using different variables

In [ ]:
SMA_1 = mean_squared_error(X_ma.iloc[1:68]["NPS_final"], X_ma.iloc[1:68]["SMA_1"])
SMA_3 = mean_squared_error(X_ma.iloc[1:68]["NPS_final"], X_ma.iloc[1:68]["SMA_3"])
SMA_1_mae = mean_absolute_error(X_ma.iloc[1:68]["NPS_final"], X_ma.iloc[1:68]["SMA_1"])
SMA_3_mae = mean_absolute_error(X_ma.iloc[1:68]["NPS_final"], X_ma.iloc[1:68]["SMA_3"])

In [ ]:
print("Error MSE SMA_1", SMA_1)
print("Error MSE SMA_3", SMA_3)
print("Error MAE SMA_1", SMA_1_mae)
print("Error  MAE SMA_3", SMA_3_mae)

Error MSE SMA_1 0.0020831853964373235
Error MSE SMA_3 0.0027336542045081342
Error MAE SMA_1 0.03634851754786886
Error  MAE SMA_3 0.041444598816468045


### 10.0 Exponential Moving averages model

This is an aplication of an exponential moving averages model 

In [ ]:
X_ma = Y.to_frame()

In [ ]:
X_ma = X_ma.append(pd.Series([np.nan]), ignore_index=True)

In [ ]:
X_ma = X_ma.drop(0,axis=1)

In [ ]:
X_ma

,NPS_final
0,0.376378
1,0.269680
2,0.201297
3,0.323791
4,0.357460
...,...
64,0.414231
65,0.279794
66,0.354576
67,0.238822


In [ ]:

X_ma['EMA_1'] = X_ma.ewm(alpha=0.1, adjust=False).mean()
X_ma['EMA_3'] = X_ma.NPS_final.ewm(alpha=0.3, adjust=False).mean()
X_ma['EMA_5'] = X_ma.NPS_final.ewm(alpha=0.5, adjust=False).mean()
X_ma['EMA_8'] = X_ma.NPS_final.ewm(alpha=0.8, adjust=False).mean()

X_ma

,NPS_final,EMA_1,EMA_3,EMA_5,EMA_8
0,0.376378,0.376378,0.376378,0.376378,0.376378
1,0.269680,0.365708,0.344368,0.323029,0.291019
2,0.201297,0.349267,0.301447,0.262163,0.219242
3,0.323791,0.346719,0.308150,0.292977,0.302881
4,0.357460,0.347793,0.322943,0.325219,0.346544
...,...,...,...,...,...
64,0.414231,0.364462,0.365990,0.376632,0.397467
65,0.279794,0.355996,0.340131,0.328213,0.303328
66,0.354576,0.355854,0.344465,0.341395,0.344326
67,0.238822,0.344150,0.312772,0.290108,0.259923


In [ ]:

colors = ['gray', 'green', 'orange',"blue","violet"]

X_ma[["NPS_final","EMA_1","EMA_3","EMA_5","EMA_8"]].plot(color=colors, linewidth=3, figsize=(12,6), alpha=0.8)


plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(labels=["Cambio NPS", "EMA - alpha=0.1", "EMA - alpha=0.3",
"EMA - alpha=0.5","EMA - alpha=0.8"], fontsize=16)

plt.title('Cambio NPS', fontsize=20)
plt.xlabel('Fecha', fontsize=16)
plt.ylabel('Cambio mensual')
plt.savefig("EMA.png")

In [ ]:
X_ma

,NPS_final,EMA_1,EMA_3,EMA_5,EMA_8
0,0.376378,0.376378,0.376378,0.376378,0.376378
1,0.269680,0.365708,0.344368,0.323029,0.291019
2,0.201297,0.349267,0.301447,0.262163,0.219242
3,0.323791,0.346719,0.308150,0.292977,0.302881
4,0.357460,0.347793,0.322943,0.325219,0.346544
...,...,...,...,...,...
64,0.414231,0.364462,0.365990,0.376632,0.397467
65,0.279794,0.355996,0.340131,0.328213,0.303328
66,0.354576,0.355854,0.344465,0.341395,0.344326
67,0.238822,0.344150,0.312772,0.290108,0.259923


In [ ]:
EMA_1 = mean_squared_error(X_ma.iloc[1:68]["NPS_final"], X_ma.iloc[1:68]["EMA_1"])
EMA_3 = mean_squared_error(X_ma.iloc[1:68]["NPS_final"], X_ma.iloc[1:68]["EMA_3"])
EMA_5 = mean_squared_error(X_ma.iloc[1:68]["NPS_final"], X_ma.iloc[1:68]["EMA_5"])
EMA_8 = mean_squared_error(X_ma.iloc[1:68]["NPS_final"], X_ma.iloc[1:68]["EMA_8"])


EMA_mae_1 = mean_absolute_error(X_ma.iloc[1:68]["NPS_final"], X_ma.iloc[1:68]["EMA_1"])
EMA_mae_3 = mean_absolute_error(X_ma.iloc[1:68]["NPS_final"], X_ma.iloc[1:68]["EMA_3"])
EMA_mae_5 = mean_absolute_error(X_ma.iloc[1:68]["NPS_final"], X_ma.iloc[1:68]["EMA_5"])
EMA_mae_8 = mean_absolute_error(X_ma.iloc[1:68]["NPS_final"], X_ma.iloc[1:68]["EMA_8"])
mean_absolute_error

<function sklearn.metrics._regression.mean_absolute_error>

In [ ]:
print("Error MSE EMA_1", EMA_1)
print("Error MSE EMA_3", EMA_3)
print("Error MSE EMA_5", EMA_5)
print("Error MSE EMA_8", EMA_8)

print("Error MAE EMA_1", EMA_mae_1)
print("Error MAE EMA_3", EMA_mae_3)
print("Error MAE EMA_5", EMA_mae_5)
print("Error MAE EMA_8", EMA_mae_8)

Error MSE EMA_1 0.00335548277151609
Error MSE EMA_3 0.00199594622213641
Error MSE EMA_5 0.0010944098773271647
Error MSE EMA_8 0.00020770048655511128
Error MAE EMA_1 0.043989363697420916
Error MAE EMA_3 0.035451994258999454
Error MAE EMA_5 0.026785615176557722
Error MAE EMA_8 0.011861928949500526


### 11.0 ARIMA model 

In [ ]:
!pip install statsmodels

In [ ]:
import statsmodels

In [ ]:
from matplotlib import pyplot

In [ ]:
# entrenar model
model = ARIMA(Y, order=(5,1,0))
model_fit = model.fit()
# resumen of fit model
print(model_fit.summary())
# grafico de los residuals
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
pyplot.show()
#plt.savefig("error_residual.png")
# grafico de densidad de residuales
residuals.plot(kind='kde')
pyplot.show()
#plt.savefig("error_densidad.png")
# stadisticas de sumario de residuals
print(residuals.describe())

                             ARIMA Model Results                              
Dep. Variable:            D.NPS_final   No. Observations:                   67
Model:                 ARIMA(5, 1, 0)   Log Likelihood                  92.909
Method:                       css-mle   S.D. of innovations              0.060
Date:                Tue, 21 Dec 2021   AIC                           -171.818
Time:                        20:51:09   BIC                           -156.385
Sample:                    05-31-2016   HQIC                          -165.711
                         - 11-30-2021                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              3.332e-05      0.002      0.016      0.987      -0.004       0.004
ar.L1.D.NPS_final    -0.7845      0.123     -6.363      0.000      -1.026      -0.543
ar.L2.D.NPS_final    -0.

In [ ]:
# dividir entre train y test
import math
X = Y.values
size = int(len(X) * 0.60)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
# validacion
for t in range(len(test)):
	model = ARIMA(history, order=(5,1,0))
	model_fit = model.fit()
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	print('predicción=%f, esperado=%f' % (yhat, obs))
# evaluar
mse = mean_squared_error(test, predictions)
mae = mean_absolute_error(test, predictions)
print('Test MSE: %.3f' % mse)
print('Test MAE: %.3f' % mae)
# graficar evaluación
fig = plt.figure()
plt.figure().clear()
plt.plot(test)
plt.plot(predictions, color='red')
plt.savefig("prediccion.png")
plt.show()


predicción=0.367567, esperado=0.388285
predicción=0.386568, esperado=0.356937
predicción=0.340128, esperado=0.424388
predicción=0.365489, esperado=0.364916
predicción=0.375505, esperado=0.336741
predicción=0.357529, esperado=0.409992
predicción=0.387829, esperado=0.280725
predicción=0.366572, esperado=0.336146
predicción=0.345052, esperado=0.375222
predicción=0.358035, esperado=0.393340
predicción=0.391020, esperado=0.376209
predicción=0.354446, esperado=0.443137
predicción=0.377534, esperado=0.361583
predicción=0.378841, esperado=0.236346
predicción=0.324754, esperado=0.453546
predicción=0.388637, esperado=0.383936
predicción=0.398089, esperado=0.467316
predicción=0.426324, esperado=0.353062
predicción=0.348496, esperado=0.275977
predicción=0.344436, esperado=0.379094
predicción=0.384738, esperado=0.367181
predicción=0.413576, esperado=0.287904
predicción=0.358948, esperado=0.392936
predicción=0.333831, esperado=0.319249
predicción=0.338831, esperado=0.414231
predicción=0.377138, espe

In [ ]:
model = ARIMA(history, order=(5,1,0))
model_fit = model.fit()
print("proxima predicción",model_fit.forecast()[0])

proxima predicción [0.33249906]


### 12.0 Creating a linear Model

#### 1.0 14-month model

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression


modelo_lineal_multivariante = LinearRegression().fit(X_train_anual, y_train_anual)

modelo_lineal_multivariante.score(X_test_anual, y_test_anual)
print(modelo_lineal_multivariante.coef_)


print(modelo_lineal_multivariante.intercept_)
y_pred = modelo_lineal_multivariante.predict(X_test_anual)


[ 2.51278445e-05 -4.24720070e-04 -2.77282346e-06 -1.68584634e-04
  8.73312851e-05  7.30929599e-04]
0.364042521599576


In [ ]:
len(y_test_anual)

14

In [ ]:
#Aqui vemos una comparación en forma de tabla  de el valor real y la predicción
y_test_anual = y_test_anual.reset_index()

y_pred = pd.DataFrame(y_pred,columns=["NPS_prediccion"])
y_autovalor_1 = pd.concat([y_pred,y_test_anual],axis=1)
y_autovalor_1

,NPS_prediccion,Año-Mes,NPS_final
0,0.371472,2020-10,0.453546
1,0.368881,2020-11,0.383936
2,0.368157,2020-12,0.467316
3,0.374169,2021-01,0.353062
4,0.365624,2021-02,0.275977
5,0.358768,2021-03,0.379094
6,0.376697,2021-04,0.367181
7,0.382033,2021-05,0.287904
8,0.372608,2021-06,0.392936
9,0.368702,2021-07,0.319249


In [ ]:
#Aqui vemos una comparación en forma de grafico  de el valor real y la predicción
plt.scatter(y_autovalor_1["NPS_prediccion"],y_autovalor_1["NPS_final"])
plt.xlabel("Predicción")
plt.ylabel("Valor Real")

Text(202.90277777777777, 0.5, 'Valor Real')

In [ ]:
y_test = y_test_anual["NPS_final"]

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error

In [ ]:
# Observamos las metricas
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
# El coeficiente de determinación: 1 es la predición perfecta
print('Mean Absolute Error: %.2f'
      % mean_absolute_error(y_test, y_pred))


Mean squared error: 0.00
Mean Absolute Error: 0.06


In [ ]:
import seaborn as sns
from matplotlib.pyplot import figure

figure(figsize=(24, 6), dpi=80)
plt.plot(dstr_anual,y_test,label="Etiqueta Real")
plt.plot(dstr_anual,y_pred,label="Etiqueta Predictiva")
plt.title("Fecha mensual contra el valor del NPS")
plt.xlabel("Fecha")
plt.ylabel("NPS")
plt.legend()
plt.show()

#### 2.0 3-month model

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression


modelo_lineal_multivariante = LinearRegression().fit(X_train_trimestral, y_train_trimestral)

modelo_lineal_multivariante.score(X_test_trimestral, y_test_trimestral)
print(modelo_lineal_multivariante.coef_)


print(modelo_lineal_multivariante.intercept_)
y_pred = modelo_lineal_multivariante.predict(X_test_trimestral)


[ 5.21836710e-04 -7.18621160e-14 -1.66912922e-04  8.85762190e-04
  4.49443963e-02  7.86247793e-03  8.08853793e-03  5.80160919e-14
 -5.41329134e-14 -2.78804757e-14 -6.70123679e-15  2.46838140e-14
  4.85245524e-14  2.13605175e-14  7.67441666e-15  7.84575640e-02
 -5.37757876e-02  6.37684350e-15  1.80027441e-01 -4.08700851e-15
  7.80278619e-15  1.74512441e-03 -3.16151158e-03 -1.28957338e-03
  2.42926933e-02  1.90092006e-02 -3.14634320e-03 -2.93119593e-01
  1.48492330e-15  1.92926357e-04 -7.10872695e-04 -1.19348975e-15
 -3.65763673e-04 -7.19103088e-04 -1.02275666e-03 -8.97982749e-03
  1.68045964e-02  5.81132364e-17 -9.08995101e-16 -1.35308431e-16
  1.35308431e-16  4.85722573e-17 -5.82867088e-16  9.71445147e-17
 -2.77555756e-17  1.02277598e-01 -9.22750222e-02  4.44089210e-16
 -9.34844936e-01 -2.22044605e-16  0.00000000e+00  4.09217249e-04
  3.28112296e-03 -9.65974378e-03  4.49750404e-03 -2.81484122e-02
 -6.51327221e-04 -9.36024968e-02  0.00000000e+00 -2.69749756e-03
  3.97770038e-06  0.00000

In [ ]:
len(y_test)

14

In [ ]:
#Aqui vemos una comparación en forma de tabla  de el valor real y la predicción
y_test_trimestral = y_test_trimestral.reset_index()
y_pred = pd.DataFrame(y_pred,columns=["NPS_prediccion"])
y_autovalor_1 = pd.concat([y_pred,y_test_trimestral],axis=1)
y_autovalor_1

,NPS_prediccion,Año-Mes,NPS_final
0,0.454159,2021-09,0.279794
1,0.382414,2021-10,0.354576
2,0.574794,2021-11,0.238822


In [ ]:
#Aqui vemos una comparación en forma de grafico  de el valor real y la predicción
plt.scatter(y_autovalor_1["NPS_prediccion"],y_autovalor_1["NPS_final"])
plt.xlabel("Predicción")
plt.ylabel("Valor Real")

Text(203.02777777777777, 0.5, 'Valor Real')

Podemos observar como la predicción del valor real de: -500 a 500 se mantienen iguales, una linea recta, luego las diferencias incrementan mediante la predicción sube. Una comprobacion de que no es un modelo lineal


In [ ]:
y_test_trimestral = y_test_trimestral["NPS_final"]

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
y_test_trimestral

0    0.279794
1    0.354576
2    0.238822
Name: NPS_final, dtype: float64

In [ ]:
# Observamos las metricas
print('Mean squared error: %.2f'
      % mean_squared_error(y_test_trimestral, y_pred))
# El coeficiente de determinación: 1 es la predición perfecta
print('Mean absolute error: %.2f'
      % mean_absolute_error(y_test_trimestral, y_pred))


Mean squared error: 0.05
Mean absolute error: 0.18


In [ ]:
import seaborn as sns
from matplotlib.pyplot import figure

figure(figsize=(24, 6), dpi=80)
plt.plot(dstr_trimestral,y_test_trimestral,label="Etiqueta Real")
plt.plot(dstr_trimestral,y_pred,label="Etiqueta Predictiva")
plt.title("Fecha mensual contra el valor del NPS")
plt.xlabel("Fecha")
plt.ylabel("NPS")
plt.legend()
plt.show()

### 13.0 Conclusion